## Fine-tuning mô hình ASR trên bộ 28k Vietnamese Voice

Notebook này sử dụng dữ liệu đã được chuẩn bị ở `prepare_data_ASR_28k.ipynb` (lưu tại `./data/asr_28k`) để fine-tune mô hình `Wav2Vec2-Base-Vietnamese-250h` cho bài toán nhận dạng tiếng nói tiếng Việt.

Các bước chính:

1. Tải lại các split `train`, `val`, `test` từ thư mục `./data/asr_28k` bằng `load_from_disk`.
2. Load mô hình và processor `nguyenvulebinh/wav2vec2-base-vietnamese-250h`.
3. Tiền xử lý:
   - Chuyển audio (`array`, `sampling_rate`) thành `input_values`.
   - Mã hóa transcript thành `labels` phù hợp với CTC.
4. Cấu hình `DataCollatorCTCWithPadding` để padding động cho batch.
5. Thiết lập `TrainingArguments` và `Trainer` để huấn luyện.
6. Huấn luyện trên tập train, đánh giá trên tập validation (metrics: WER, CER).
7. Đánh giá cuối trên tập test.
8. Lưu mô hình đã fine-tune vào `./models/wav2vec2-finetuned-28k-final`.


In [ ]:
# Cài đặt thư viện cần thiết (chạy một lần trong môi trường mới)
# Nếu môi trường đã có các thư viện này, có thể bỏ qua cell này
%pip install -q transformers datasets jiwer

from datasets import load_from_disk
from transformers import (
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    TrainingArguments,
    Trainer,
    DataCollatorCTCWithPadding,
)
from pathlib import Path
from jiwer import wer, cer
import numpy as np
import torch

# Kiểm tra thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Thiết bị sử dụng:", device)

# 1. Tải dữ liệu đã chuẩn bị
print("\n[1/6] Đang tải dữ liệu ASR từ ./data/asr_28k ...")
DATA_DIR = Path("./data/asr_28k")
train_dataset = load_from_disk(str(DATA_DIR / "train"))
val_dataset = load_from_disk(str(DATA_DIR / "val"))
test_dataset = load_from_disk(str(DATA_DIR / "test"))

print("Số mẫu:")
print("  Train:", len(train_dataset))
print("  Val:  ", len(val_dataset))
print("  Test: ", len(test_dataset))

# 2. Load mô hình và processor Wav2Vec2
print("\n[2/6] Đang tải mô hình Wav2Vec2-Base-Vietnamese-250h...")
MODEL_ID = "nguyenvulebinh/wav2vec2-base-vietnamese-250h"
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)
model.to(device)

print("Đã tải mô hình:", MODEL_ID)
print("Kích thước vocab:", model.config.vocab_size)

# 3. Tiền xử lý dữ liệu: audio → input_values, transcript → labels
print("\n[3/6] Đang tiền xử lý dữ liệu (audio → input_values, transcript → labels)...")

def prepare_batch(batch):
    """Tiền xử lý 1 batch cho Wav2Vec2 CTC."""
    audio = batch["audio"]
    # audio["array"]: numpy array 1D, audio["sampling_rate"]: tần số lấy mẫu
    inputs = processor(
        audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_tensors="pt",
    )
    batch["input_values"] = inputs.input_values[0]

    # Mã hóa transcript thành labels
    with processor.as_target_processor():
        labels = processor(batch["transcript"]).input_ids
    batch["labels"] = labels
    return batch

# Áp dụng cho train và val (test sẽ xử lý riêng khi đánh giá nếu cần)
train_proc = train_dataset.map(
    prepare_batch,
    remove_columns=train_dataset.column_names,
    num_proc=4,
)
val_proc = val_dataset.map(
    prepare_batch,
    remove_columns=val_dataset.column_names,
    num_proc=4,
)

print("Ví dụ 1 sample sau tiền xử lý:")
print("  input_values shape:", train_proc[0]["input_values"].shape)
print("  labels length:", len(train_proc[0]["labels"]))

# 4. Data collator cho CTC
print("\n[4/6] Thiết lập DataCollatorCTCWithPadding...")

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

# 5. Hàm tính toán metrics (WER, CER)
print("\n[5/6] Định nghĩa hàm compute_metrics (WER, CER)...")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    # Thay -100 (label padding) bằng token pad để decode
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(label_ids, group_tokens=False)

    wer_score = wer(label_str, pred_str)
    cer_score = cer(label_str, pred_str)

    return {"wer": wer_score, "cer": cer_score}

# 6. Thiết lập TrainingArguments và Trainer
print("\n[6/6] Thiết lập TrainingArguments và Trainer...")

OUTPUT_DIR = Path("./models/wav2vec2-finetuned-28k")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

training_args = TrainingArguments(
    output_dir=str(OUTPUT_DIR),
    group_by_length=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    save_strategy="steps",
    num_train_epochs=3,
    fp16=torch.cuda.is_available(),
    learning_rate=3e-4,
    warmup_steps=500,
    logging_steps=100,
    eval_steps=1000,
    save_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_proc,
    eval_dataset=val_proc,
    tokenizer=processor.feature_extractor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Thiết lập Trainer xong. Sẵn sàng huấn luyện.")
print("\nGọi trainer.train() trong cell tiếp theo để bắt đầu fine-tuning.")


In [ ]:
# Bắt đầu fine-tuning ASR
print("Bắt đầu fine-tuning mô hình ASR trên bộ 28k...")
train_result = trainer.train()

print("\nKết quả huấn luyện (tóm tắt):")
print(train_result)

# Đánh giá trên tập validation cuối cùng
print("\nĐánh giá trên tập validation:")
val_metrics = trainer.evaluate()
print(val_metrics)

# Chuẩn bị test set cho đánh giá cuối
print("\nTiền xử lý test set để đánh giá cuối cùng...")

test_proc = test_dataset.map(
    prepare_batch,
    remove_columns=test_dataset.column_names,
    num_proc=4,
)

print("Số mẫu test:", len(test_proc))

print("\nĐánh giá trên tập test:")
test_metrics = trainer.evaluate(test_proc)
print(test_metrics)

# Lưu mô hình fine-tuned
FINAL_DIR = Path("./models/wav2vec2-finetuned-28k-final")
FINAL_DIR.mkdir(parents=True, exist_ok=True)

print("\nĐang lưu mô hình đã fine-tune vào:", FINAL_DIR.resolve())
trainer.save_model(str(FINAL_DIR))
processor.save_pretrained(str(FINAL_DIR))

print("Hoàn thành fine-tuning ASR và lưu mô hình.")
